In [294]:
import pandas as pd
import numpy as np
from functools import reduce
from fbprophet import Prophet
import matplotlib.pyplot as plt
from csv import writer

plt.style.use('fivethirtyeight')

In [354]:
# Directory path
path = "C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\new_files\\unit_sales\\*.csv"
path1 = "C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\new_files\\mape results\\unit_mape\\"

In [355]:
#read data
cust_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_customers_dataset.csv')
geo_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_geolocation_dataset.csv')
orders_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_orders_dataset.csv')
order_item_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_order_items_dataset.csv')
order_payments_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_order_payments_dataset.csv')
order_reviews_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_order_reviews_dataset.csv')
product_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_products_dataset.csv')
sellers_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\olist_sellers_dataset.csv')
category_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\Brazilian Eccomerce Dataset\\product_category_name_translation.csv')

In [356]:
merged_orders = pd.merge(orders_data,order_item_data, on='order_id')
merged_orders.nunique()

order_id                         98666
customer_id                      98666
order_status                         7
order_purchase_timestamp         98112
order_approved_at                90174
order_delivered_carrier_date     81017
order_delivered_customer_date    95664
order_estimated_delivery_date      450
order_item_id                       21
product_id                       32951
seller_id                         3095
shipping_limit_date              93318
price                             5968
freight_value                     6999
dtype: int64

In [357]:
merged_products = pd.merge(merged_orders, product_data, on ='product_id')
merged_products.nunique()

order_id                         98666
customer_id                      98666
order_status                         7
order_purchase_timestamp         98112
order_approved_at                90174
order_delivered_carrier_date     81017
order_delivered_customer_date    95664
order_estimated_delivery_date      450
order_item_id                       21
product_id                       32951
seller_id                         3095
shipping_limit_date              93318
price                             5968
freight_value                     6999
product_category_name               73
product_name_lenght                 66
product_description_lenght        2960
product_photos_qty                  19
product_weight_g                  2204
product_length_cm                   99
product_height_cm                  102
product_width_cm                    95
dtype: int64

In [358]:
merged_products.shape

(112650, 22)

In [359]:
merged_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112650 non-null  object 
 1   customer_id                    112650 non-null  object 
 2   order_status                   112650 non-null  object 
 3   order_purchase_timestamp       112650 non-null  object 
 4   order_approved_at              112635 non-null  object 
 5   order_delivered_carrier_date   111456 non-null  object 
 6   order_delivered_customer_date  110196 non-null  object 
 7   order_estimated_delivery_date  112650 non-null  object 
 8   order_item_id                  112650 non-null  int64  
 9   product_id                     112650 non-null  object 
 10  seller_id                      112650 non-null  object 
 11  shipping_limit_date            112650 non-null  object 
 12  price                         

In [360]:
print (merged_products.order_purchase_timestamp.min())
print (merged_products.order_purchase_timestamp.max())

2016-09-04 21:15:19
2018-09-03 09:06:57


In [361]:
merged_products.shape

(112650, 22)

In [362]:
pd.set_option("display.max_rows",100000)

In [363]:
date_split1= merged_products['order_purchase_timestamp'].str.split(' ')
merged_products['order_purchase_date']=date_split1.str.get(0)

In [364]:
merged_products.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_purchase_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,2017-10-02
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,...,7.78,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,2017-08-15
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,...,7.78,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,2017-08-02
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13 00:00:00,1,87285b34884572647811a353c7ac498a,...,14.10,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,2017-10-23
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,...,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,2018-07-24


In [365]:
merged_products.shape

(112650, 23)

In [310]:
merged_products2.to_csv("C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\unit_sales\\input\\unit_sales_merged_daily.csv", index=False)

#### Extraction function

In [311]:
a = merged_products2['product_category_name'].unique()

In [312]:
a

array(['moveis_decoracao', 'telefonia', 'beleza_saude', 'bebes',
       'brinquedos', 'esporte_lazer', 'fashion_calcados',
       'relogios_presentes', 'automotivo', 'cama_mesa_banho',
       'climatizacao', 'consoles_games', 'cool_stuff',
       'ferramentas_jardim', 'informatica_acessorios',
       'moveis_escritorio', 'perfumaria', 'telefonia_fixa',
       'utilidades_domesticas', 'eletronicos', 'fashion_roupa_feminina',
       'fashion_roupa_masculina', 'industria_comercio_e_negocios',
       'livros_tecnicos', 'market_place', 'fashion_bolsas_e_acessorios',
       'pet_shop', 'audio', 'fraldas_higiene', 'livros_interesse_geral',
       'alimentos', 'moveis_quarto', 'musica', 'papelaria',
       'eletroportateis', 'eletrodomesticos_2',
       'fashion_underwear_e_moda_praia', 'agro_industria_e_comercio',
       'casa_conforto_2', 'malas_acessorios',
       'moveis_cozinha_area_de_servico_jantar_e_jardim',
       'seguros_e_servicos', 'instrumentos_musicais', 'moveis_sala',
       'c

In [313]:
list_df = ['beleza_saude', 'moveis_decoracao', 'telefonia', 'alimentos',
       'audio', 'automotivo', 'bebes', 'brinquedos', 'cama_mesa_banho',
       'climatizacao', 'consoles_games', 'cool_stuff', 'eletronicos',
       'esporte_lazer', 'fashion_bolsas_e_acessorios', 'fashion_calcados',
       'fashion_roupa_feminina', 'fashion_roupa_masculina',
       'ferramentas_jardim', 'fraldas_higiene',
       'industria_comercio_e_negocios', 'informatica_acessorios',
       'livros_interesse_geral', 'livros_tecnicos', 'market_place',
       'moveis_escritorio', 'perfumaria', 'pet_shop',
       'relogios_presentes', 'telefonia_fixa', 'utilidades_domesticas',
       'agro_industria_e_comercio', 'casa_conforto_2',
       'construcao_ferramentas_construcao', 'dvds_blu_ray',
       'eletrodomesticos_2', 'eletroportateis',
       'fashion_underwear_e_moda_praia', 'instrumentos_musicais',
       'malas_acessorios',
       'moveis_cozinha_area_de_servico_jantar_e_jardim', 'moveis_quarto',
       'moveis_sala', 'musica', 'papelaria', 'pcs', 'seguros_e_servicos',
       'casa_conforto', 'casa_construcao', 'cine_foto',
       'construcao_ferramentas_jardim',
       'construcao_ferramentas_seguranca', 'eletrodomesticos',
       'livros_importados', 'tablets_impressao_imagem',
       'alimentos_bebidas', 'artes', 'bebidas', 'fashion_esporte',
       'sinalizacao_e_seguranca', 'artigos_de_natal',
       'construcao_ferramentas_ferramentas', 'la_cuisine',
       'moveis_colchao_e_estofado', 'pc_gamer', 'artes_e_artesanato',
       'fashion_roupa_infanto_juvenil', 'portateis_casa_forno_e_cafe',
       'artigos_de_festas', 'cds_dvds_musicais',
       'construcao_ferramentas_iluminacao', 'flores',
       'portateis_cozinha_e_preparadores_de_alimentos']

In [314]:
# Fuction to extract data on product categories
def extract_df(list_df, merged_products2):
    for i in list_df:
        
        new_df = merged_products2[merged_products2['product_category_name'] == i]
        new_df.to_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\unit_sales\\extracted_files\\'+str(i)+'.csv', index=False)
        print("length of " +str(i)+":", len(new_df))
        
extract_df(list_df, merged_products2)

length of beleza_saude: 602
length of moveis_decoracao: 607
length of telefonia: 588
length of alimentos: 253
length of audio: 235
length of automotivo: 582
length of bebes: 562
length of brinquedos: 590
length of cama_mesa_banho: 598
length of climatizacao: 198
length of consoles_games: 441
length of cool_stuff: 584
length of eletronicos: 511
length of esporte_lazer: 603
length of fashion_bolsas_e_acessorios: 542
length of fashion_calcados: 188
length of fashion_roupa_feminina: 37
length of fashion_roupa_masculina: 93
length of ferramentas_jardim: 585
length of fraldas_higiene: 27
length of industria_comercio_e_negocios: 153
length of informatica_acessorios: 593
length of livros_interesse_geral: 307
length of livros_tecnicos: 172
length of market_place: 220
length of moveis_escritorio: 454
length of perfumaria: 580
length of pet_shop: 524
length of relogios_presentes: 571
length of telefonia_fixa: 179
length of utilidades_domesticas: 587
length of agro_industria_e_comercio: 144
length

#### Modeling using Prophet on unit sales

In [340]:
import pandas as pd
import numpy as np
from functools import reduce
from fbprophet import Prophet
import matplotlib.pyplot as plt
from csv import writer
import glob

plt.style.use('fivethirtyeight')

In [341]:
# Directory path
path = "C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\unit_sales\\extracted_files\\*.csv"
path1 = "C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\unit_sales\\mape results\\"

In [352]:
# Function to forecast the data on monthly frequency
def future_prediction(path, path1):
    # Iterate through input files
    for fname in glob.glob(path):
        forecast_df=pd.read_csv(fname)
        df1 = forecast_df.copy()
        
        
        # Converting df into 'ds','y' format for forecasting
        forecast_df['order_purchase_date'] = forecast_df['order_purchase_date'].str[:7]
        forecast_df= forecast_df.groupby(['order_purchase_date', 'product_category_name']).size().reset_index(name='Total Unit Sales')
        forecast_df = forecast_df[['order_purchase_date','Total Unit Sales']]
        forecast_df = forecast_df.rename(columns = {'order_purchase_date':'ds','Total Unit Sales':'y'})   
        #forecast_df = forecast_df.groupby('ds')['y'].mean().reset_index()
        forecast_df['ds']=pd.to_datetime(forecast_df['ds'], errors='coerce')
        
        #m = Prophet(yearly_seasonality=False)
        
        m = Prophet(seasonality_mode='multiplicative')
        
        m.fit(forecast_df)
        future = m.make_future_dataframe(periods=12, freq='MS')
        forecast_ndays = m.predict(future)
        data_frames = [forecast_df,forecast_ndays]
        forecast_data_ndays = reduce(lambda left,right: pd.merge(left,right, on=['ds'],how='outer'),data_frames)
        forecast_data1_ndays = forecast_data_ndays.loc[:,['ds','y','yhat','yhat_lower','yhat_upper']]
        forecast_data1_ndays1 = forecast_data1_ndays.iloc[:-12,:]
        err = np.mean(np.abs((forecast_data1_ndays.y - forecast_data1_ndays.yhat) / forecast_data1_ndays.y)) * 100
        forecast_data1_ndays['mape'] = err
        forecast_data1_ndays['product_category_name']=df1['product_category_name']
        a = forecast_data1_ndays['product_category_name'].iloc[0]
        forecast_data1_ndays = forecast_data1_ndays.rename(columns = {'ds':'order_purchase_date','y':'Total Unit Sales'})
        forecast_data1_ndays = forecast_data1_ndays[['order_purchase_date','product_category_name','Total Unit Sales','yhat','mape']]
        b = str(err)
        
        list1 = []
        list1.append(a)
        list1.append(b)
        print(list1)
        forecast_data1_ndays.to_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\unit_sales\\results\\'+a+'_results'+'.csv', index=False)
        # Saving the mape resukts into the file
        with open(path1+'mape_results.csv', 'a',newline='') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(list1)
            f_object.close() 

print("************************ forecast analysis *********************************")  

************************ forecast analysis *********************************


In [353]:
future_prediction(path, path1)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 14.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


['agro_industria_e_comercio', '95.89183405588567']


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 13.


['alimentos', '68.8731664113329']


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 13.


...

['tablets_impressao_imagem', '60.28367930251854']


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


['telefonia', '97.37179131224241']


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


['telefonia_fixa', '30.098871675496714']
['utilidades_domesticas', '24.8179279112872']
